In [7]:
!pip install pandas scikit-learn nltk

In [8]:
from google.colab import files
uploaded = files.upload()

Saving ticket_helpdesk_labeled_multi_languages_english_spain_french_german.csv to ticket_helpdesk_labeled_multi_languages_english_spain_french_german (3).csv


In [9]:
import pandas as pd
import re
import nltk

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df = pd.read_csv("ticket_helpdesk_labeled_multi_languages_english_spain_french_german.csv")
df.head()


,queue,priority,software_used,hardware_used,accounting_category,language,subject,text
0,Hardware,2,NaN,Wireless Mouse,NaN,en,Wireless Mouse suddenly stops working,"Dear Support Team, I've been using the Wireles..."
1,Hardware,2,NaN,IP PBX,NaN,fr,Problème de connexions IP PBX,"Bonjour, nous rencontrons un problème avec not..."
2,Hardware,2,NaN,SFX-Netzteil,NaN,de,Problem mit meinem SFX-Netzteil,"Sehr geehrte Damen und Herren, mein SFX-Netzte..."
3,Accounting,2,NaN,NaN,Customer Inquiries::Technical Support,en,Invoice Adjustment Request,"Dear Customer Support,\nI recently received my..."
4,Software,2,Arbitrum,NaN,NaN,en,Issue with Arbitrum: UI not loading,"Hello Support Team,\nI've been experiencing an..."


In [11]:
df.columns


Index(['queue', 'priority', 'software_used', 'hardware_used',
       'accounting_category', 'language', 'subject', 'text'],
      dtype='object')

In [13]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["text"].apply(clean_text)

In [15]:
X = df["clean_text"]
y = df["queue"] # Using 'queue' as the label column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [17]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)


LogisticRegression(max_iter=1000)

In [18]:
y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.625
              precision    recall  f1-score   support

  Accounting       1.00      0.33      0.50         9
    Hardware       0.69      0.64      0.67        14
    Software       0.54      0.76      0.63        17

    accuracy                           0.62        40
   macro avg       0.74      0.58      0.60        40
weighted avg       0.70      0.62      0.62        40



In [19]:
new_ticket = ["My app is not working and payment failed"]
new_ticket_clean = [clean_text(new_ticket[0])]
new_ticket_vec = vectorizer.transform(new_ticket_clean)

prediction = model.predict(new_ticket_vec)
print("Predicted Category:", prediction[0])


Predicted Category: Hardware
